In [23]:
# Load required packages
import pandas as pd
import sklearn as sk
from sklearn import preprocessing
import tensorflow as tf
import numpy as np
import scipy as sp
import os
import threading
import multiprocessing
import pickle
import keras


print "Packages loaded"


Packages loaded


In [5]:
### Functions for feature extraction
class arayi(object):
    def __init__(self):
        pass
        
    # Get list of tracks for extraction
    def getTrackList(self,pathToTrack,listBool,trackName):
        if listBool == False:
            self.filelist=[]
            with open(pathToTrack+"/metadata/perfs.csv","r") as perfData:
                perfData.next()
                for line in perfData:
                    line=line.split(",")
                    if (line[2].replace("_"," ").lstrip().rstrip() == trackName and line[6].rstrip() == "EN"):
                        self.filelist.append(line[1])
            self.filelist=tuple(self.filelist)
            return self.filelist
        else:
            self.filelist = os.listdir(pathToTrack)            
            return self.list

    def getLabelList(self,pathToTrack,listBool,trackName):
        if listBool == False:
            self.filelist=[]
            with open(pathToTrack+"/metadata/perfs.csv","r") as perfData:
                perfData.next()
                for line in perfData:
                    line=line.split(",")
                    if (line[2].replace("_"," ").lstrip().rstrip() == trackName and line[6].rstrip() == "EN"):
                        self.filelist.append(line[5])
            self.filelist=tuple(self.filelist)
            return self.filelist
        else:
            self.filelist = os.listdir(pathToTrack)            
            return self.list

    # Extract Melodic Contours from Dataset
    def extractMelody(self, pathToTrack,trackList):
        self.extractStatus = 0.0
        pathToTrack = pathToTrack+"raw/"
        # Extracts contour for a track snd saves output for each track
        for track in trackList:
            self.trackOut = "/Users/nus/singability_data/contours/%s.yml" % track.split(".")[0]
            os.system("essentia_streaming_predominantpitchmelodia %s %s" % (pathToTrack+track+".m4a", self.trackOut))
            if float(trackList.index(track)+1)/len(trackList) > self.extractStatus:
                print "Extracting...%s %% complete." % (self.extractStatus*100)
                self.extractStatus += 0.33
                
    def cleanTrack(self,labelList):
        df = pickle.load(open("/Users/nus/singability_data/letItGo.pkl","rb"))
        df=df.transpose()
        self.lList=[]
        for col in df:
            numNull=df[col].isnull().sum()
            if numNull > 60:
                df.drop(col,1,inplace=True)
                self.lList.append(col)
        df.fillna(0)
        labelList=[x for i,x in enumerate(labelList) if i not in lList]
        return (df,labelList)


In [3]:
## Threaded feature extraction - Needs work
def threadMelodyExtract(self,pathToTrack,trackList):
    ##TODO: Threading locks
    threads = []
    t=[]
    for track in trackList:
        if ".m4a" not in track or "_" not in track:
            continue
        t = threading.Thread(target=a.extractMelody, args=(pathToTrack,track,))
        threads.append(t)
        if len(threads) > 9:
            t.start()
            t.join()
            t=[]
            threads=[]

# def threadMelodyExtract2(self,pathToTrack,trackList):
#     ##TODO: Threading locks
#     lock = threading.Lock()
#     threads = []
#     for track in trackList:
#         if ".m4a" not in track or "_" not in track:
#             continue
#         t = threading.Thread(target=a.extractMelody, args=(pathToTrack,track,))
#         lock.acquire()
#         if not lock.acquire(False):
#             print "Fail"
#         else:
#             try:
#                 t.start()
#                 t.join()
#         finally:
#             lock.release()
            
# arayi.threadMelodyExtract = threadMelodyExtract
# arayi.threadMelodyExtract2 = threadMelodyExtract2

In [5]:
def loadContourData(self,pathToContours):
    cList=os.listdir(pathToContours)
    self.contourMatrix = []
    for contour in cList:
        if ".yml" not in contour:
            continue
        x=list(open(pathToContours+contour,"r"))
        x=map(float,x[7].lstrip().replace("pitch: ","").replace("[","").replace("]","").split(","))
        self.contourMatrix.append(x)
    df = pd.DataFrame(self.contourMatrix,columns=None)
    return df

def excludeData(self,contourList):
    # THIS FUNCTION WILL CHECK ALL OF THE COMPLETED CONTOURS TO A LIST OF APPLICABLE SONGS.
    pass

arayi.loadContourData = loadContourData
a=arayi()
df=a.loadContourData("/Users/nus/singability_data/contours/")

In [26]:
#df.to_pickle("/Users/nus/Documents/Github/Singability/letItGo.pkl")
# df = pickle.load(open("/Users/nus/singability_data/letItGo.pkl","rb"))
# print "loaded"
# df=df.transpose()
# dflist=range(0,100)
# dflist
# df.iloc[:,dflist].to_csv(open("/Users/nus/Documents/GitHub/Singability/contours.csv","w"))
# for index, row in df.iterrows():
#     row=[0.0 if x != "NaN" else x for x in row]
#     print index,len(row)
#     if index > 10:
#         break

# ##### TO DO: SOME TRACKS ARE INCOMPLETE AND CONTAIN NAN VALUES. MAXIMUM SAMPLE TO BE REMOVED = 63682
# nullDict={}
# for col in df:
#     numNull=df[col].isnull().sum()
#     if numNull > 60:
#         df.drop(col,1,inplace=True)
#     if numNull not in nullDict.keys():
#         nullDict[numNull]=1
#     else:
#         nullDict[numNull]+=1

# for y in sorted(nullDict):
#     print y, nullDict[y]
# print sum(nullDict.values())

#df=df.fillna(0)
df

x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)
df

,0,1,2,3,4,5,6,7,8,9,...,1615,1616,1617,1618,1619,1620,1621,1622,1623,1624
0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.318638,0.0,...,0.523645,0.000000,0.269493,0.267941,0.000000,0.216382,0.445447,0.0,0.000000,0.000000
1,0.740548,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.314978,0.0,...,0.526678,0.000000,0.269493,0.267941,0.000000,0.215136,0.442882,0.0,0.000000,0.000000
2,0.554783,0.000000,0.0,0.0,0.0,0.000000,0.0,0.314978,0.311360,0.0,...,0.523645,0.000000,0.269493,0.267941,0.000000,0.216382,0.442882,0.0,0.574347,0.000000
3,0.554783,0.000000,0.0,0.0,0.0,0.703033,0.0,0.314978,0.311360,0.0,...,0.523645,0.000000,0.269493,0.267941,0.000000,0.215136,0.442882,0.0,0.577674,0.000000
4,0.545252,0.000000,0.0,0.0,0.0,0.707105,0.0,0.316803,0.306011,0.0,...,0.520629,0.000000,0.269493,0.267941,0.000000,0.215136,0.442882,0.0,0.581021,0.000000
5,0.526679,0.000000,0.0,0.0,0.0,0.707105,0.0,0.320484,0.302496,0.0,...,0.520629,0.000000,0.272625,0.264864,0.000000,0.215136,0.442882,0.0,0.577674,0.000000
6,0.542111,0.000000,0.0,0.0,0.0,0.711202,0.0,0.326086,0.299022,0.0,...,0.517630,0.000000,0.272625,0.258814,0.000000,0.215136,0.440331,0.0,0.561229,0.000000
7,0.535885,0.000000,0.0,0.0,0.0,0.715322,0.0,0.327975,0.295587,0.0,...,0.517630,0.195014,0.274204,0.251446,0.000000,0.213897,0.440331,0.0,0.574347,0.000000
8,0.514649,0.000000,0.0,0.0,0.0,0.715322,0.0,0.329875,0.292192,0.0,...,0.514649,0.197280,0.274204,0.245703,0.000000,0.215136,0.440331,0.0,0.577674,0.000000
9,0.514649,0.000000,0.0,0.0,0.0,0.715322,0.0,0.337585,0.290509,0.0,...,0.508738,0.199573,0.274204,0.244288,0.000000,0.215136,0.440331,0.0,0.591177,0.000000


In [25]:
# Run Functions
# pathToTrack = "/Users/nus/singability_data/"
# a = arayi()
# tList = a.getTrackList(pathToTrack, False, "let it go")
# lList = a.getLabelList(pathToTrack, False, "let it go")
# print len(lList), len(tList)
# a.extractMelody(pathToTrack, tList)
#songDf,songLabels=a.cleanTrack(lList)
df

# a.threadMelodyExtract(pathToTrack,tList)

'''
NUM_WORKERS = 4
processes = [multiprocessing.Process(target=a.extractMelody(pathToTrack,tList)) for _ in range(NUM_WORKERS)]
print "Starting Process"
[process.start() for process in processes]
print "Joining process"
[process.join() for process in processes]
'''


'\nNUM_WORKERS = 4\nprocesses = [multiprocessing.Process(target=a.extractMelody(pathToTrack,tList)) for _ in range(NUM_WORKERS)]\nprint "Starting Process"\n[process.start() for process in processes]\nprint "Joining process"\n[process.join() for process in processes]\n'